In [2]:
import pandas as pd
data = pd.read_csv('fake.csv', error_bad_lines=False);


In [3]:
data_text = data[['text']]
data_text.index

RangeIndex(start=0, stop=12999, step=1)

In [4]:
data_text["index"] = data_text.index


C:\Users\vidya\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [5]:
documents = data_text

In [6]:
print(len(documents))
print(documents[:5])

12999
                                                text  index
0  Print They should pay all the back all the mon...      0
1  Why Did Attorney General Loretta Lynch Plead T...      1
2  Red State : \nFox News Sunday reported this mo...      2
3  Email Kayla Mueller was a prisoner and torture...      3
4  Email HEALTHCARE REFORM TO MAKE AMERICA GREAT ...      4


In [1]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\vidya\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


True

In [ ]:
!pip install -U genism

In [ ]:
!conda install -c anaconda gensim

In [7]:
print(WordNetLemmatizer().lemmatize('went', pos='v'))

go


In [8]:
stemmer = SnowballStemmer('english')
original_words = ['caresses', 'flies', 'dies', 'mules', 'denied','died', 'agreed', 'owned', 
           'humbled', 'sized','meeting', 'stating', 'siezing', 'itemization','sensational', 
           'traditional', 'reference', 'colonizer','plotted']
singles = [stemmer.stem(plural) for plural in original_words]
pd.DataFrame(data = {'original word': original_words, 'stemmed': singles})

,original word,stemmed
0,caresses,caress
1,flies,fli
2,dies,die
3,mules,mule
4,denied,deni
5,died,die
6,agreed,agre
7,owned,own
8,humbled,humbl
9,sized,size


In [9]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result


In [14]:
documents = documents.dropna(subset=['text'])

In [15]:


doc_sample = documents[documents['index'] == 4310].values[0][0]

print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['Common', 'Sense', 'as', 'defined', 'by', 'the', 'Cambridge', 'Academic', 'Text', 'Dictionary', 'is', 'the', 'ability', 'to', 'use', 'good', 'judgement', 'in', 'making', 'decisions', 'to', 'live', 'in', 'a', 'reasonable', 'and', 'safe', 'way.', 'While', 'that', 'may', 'have', 'sounded', 'reasonable', 'and', 'safe', 'back', 'in', 'the', 'sixties', 'and', 'seventies', 'when', 'I', 'was', 'growing', 'up,', 'I', 'find', 'myself', 'at', 'risk', 'daily', 'based', 'on', 'the', 'judgement', 'of', 'others', 'who', 'make', 'decisions', 'that', 'have', 'lasting', 'and', 'irreversible', 'consequences.', 'Ever', 'since', 'I', 'started', 'writing', 'sporadically,', 'beginning', 'in', 'the', 'early', 'eighties', 'for', 'small', 'newspapers', 'like', 'the', 'Chicago', 'Daily', 'News', 'and', 'the', 'Chicago', 'Defender,', 'I', 'have', 'enjoyed', 'the', 'opportunity', 'to', 'research,', 'find', 'and', 'expose', 'resources.', 'I', 'started', 'with', '“Canons', 'on', 'Careers”,', 'wh

In [16]:

processed_docs = documents['text'].map(preprocess)


In [17]:
processed_docs[:10]

0    [print, money, plus, entir, famili, come, need...
1    [attorney, general, loretta, lynch, plead, fif...
2    [state, news, sunday, report, morn, anthoni, w...
3    [email, kayla, mueller, prison, tortur, isi, c...
4    [email, healthcar, reform, america, great, mar...
5    [print, hillari, go, absolut, berserk, explod,...
6    [break, nypd, readi, arrest, weiner, case, hil...
7    [break, nypd, readi, arrest, weiner, case, hil...
8    [limbaugh, say, revel, wikileak, materi, start...
9    [email, peopl, sick, evil, stop, law, mean, me...
Name: text, dtype: object

In [18]:

dictionary = gensim.corpora.Dictionary(processed_docs)

In [19]:

count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 asap
1 benefit
2 bust
3 case
4 come
5 commit
6 control
7 deport
8 entir
9 famili
10 fraud


In [20]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [21]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[4310]

[(3, 1),
 (4, 1),
 (5, 1),
 (14, 1),
 (15, 2),
 (24, 1),
 (36, 1),
 (79, 1),
 (106, 2),
 (119, 1),
 (125, 4),
 (126, 3),
 (132, 1),
 (134, 1),
 (151, 1),
 (152, 1),
 (160, 1),
 (167, 1),
 (191, 2),
 (195, 1),
 (199, 1),
 (206, 2),
 (211, 2),
 (214, 1),
 (216, 1),
 (217, 1),
 (223, 1),
 (225, 1),
 (226, 2),
 (237, 1),
 (247, 1),
 (249, 1),
 (257, 1),
 (265, 2),
 (270, 5),
 (271, 1),
 (281, 1),
 (306, 1),
 (324, 3),
 (351, 3),
 (371, 1),
 (380, 1),
 (381, 1),
 (408, 1),
 (417, 1),
 (420, 1),
 (423, 1),
 (455, 1),
 (457, 3),
 (459, 1),
 (460, 1),
 (478, 1),
 (479, 1),
 (517, 1),
 (520, 3),
 (523, 1),
 (535, 1),
 (561, 1),
 (570, 4),
 (610, 2),
 (619, 1),
 (653, 1),
 (656, 1),
 (670, 1),
 (688, 1),
 (692, 1),
 (711, 2),
 (754, 2),
 (761, 1),
 (778, 1),
 (779, 1),
 (798, 1),
 (801, 2),
 (808, 2),
 (809, 1),
 (817, 1),
 (846, 1),
 (847, 1),
 (852, 2),
 (855, 1),
 (862, 1),
 (865, 1),
 (871, 3),
 (916, 1),
 (926, 1),
 (933, 1),
 (947, 1),
 (953, 1),
 (964, 1),
 (977, 1),
 (1010, 1),
 (1024, 1

In [23]:
bow_doc_4310 = bow_corpus[4310]

for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                                     dictionary[bow_doc_4310[i][0]], 
                                                     bow_doc_4310[i][1]))


Word 3 ("case") appears 1 time.
Word 4 ("come") appears 1 time.
Word 5 ("commit") appears 1 time.
Word 14 ("like") appears 1 time.
Word 15 ("million") appears 2 time.
Word 24 ("relat") appears 1 time.
Word 36 ("american") appears 1 time.
Word 79 ("inform") appears 1 time.
Word 106 ("public") appears 2 time.
Word 119 ("state") appears 1 time.
Word 125 ("world") appears 4 time.
Word 126 ("write") appears 3 time.
Word 132 ("base") appears 1 time.
Word 134 ("break") appears 1 time.
Word 151 ("form") appears 1 time.
Word 152 ("give") appears 1 time.
Word 160 ("know") appears 1 time.
Word 167 ("news") appears 1 time.
Word 191 ("donald") appears 2 time.
Word 195 ("great") appears 1 time.
Word 199 ("presid") appears 1 time.
Word 206 ("trump") appears 2 time.
Word 211 ("america") appears 2 time.
Word 214 ("bring") appears 1 time.
Word 216 ("busi") appears 1 time.
Word 217 ("care") appears 1 time.
Word 223 ("complet") appears 1 time.
Word 225 ("cost") appears 1 time.
Word 226 ("countri") appears

In [24]:

from gensim import corpora, models

tfidf = models.TfidfModel(bow_corpus)

In [25]:
corpus_tfidf = tfidf[bow_corpus]

In [26]:
from pprint import pprint

for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.3500512547230274),
 (1, 0.14779052811571117),
 (2, 0.2509550132368066),
 (3, 0.0959276381032166),
 (4, 0.05650432146213837),
 (5, 0.13795848967793725),
 (6, 0.09847959217082528),
 (7, 0.23881625132641596),
 (8, 0.11286182174525332),
 (9, 0.11115771642046168),
 (10, 0.16957916134677647),
 (11, 0.14508239585282315),
 (12, 0.18489604253976888),
 (13, 0.16170327437243945),
 (14, 0.04460067252936085),
 (15, 0.09505506146952088),
 (16, 0.11241507548990382),
 (17, 0.09590387754528427),
 (18, 0.16072623307835415),
 (19, 0.07263567932123731),
 (20, 0.17496626609002672),
 (21, 0.19336028975000125),
 (22, 0.15940061131640726),
 (23, 0.19077058991862686),
 (24, 0.09364517477467808),
 (25, 0.060942405994516446),
 (26, 0.2583702830532261),
 (27, 0.3500512547230274),
 (28, 0.33623480159032987),
 (29, 0.21990575284339772),
 (30, 0.04787518694858638)]


In [27]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

In [28]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.011*"elect" + 0.011*"trump" + 0.011*"state" + 0.010*"vote" + 0.009*"clinton" + 0.006*"peopl" + 0.005*"presid" + 0.005*"voter" + 0.005*"obama" + 0.004*"time"
Topic: 1 
Words: 0.007*"bank" + 0.006*"clinton" + 0.006*"world" + 0.006*"state" + 0.005*"gold" + 0.005*"money" + 0.005*"peopl" + 0.004*"trump" + 0.004*"year" + 0.004*"nation"
Topic: 2 
Words: 0.024*"clinton" + 0.014*"hillari" + 0.011*"email" + 0.007*"investig" + 0.006*"campaign" + 0.006*"state" + 0.005*"report" + 0.005*"presid" + 0.004*"trump" + 0.004*"elect"
Topic: 3 
Words: 0.006*"report" + 0.005*"year" + 0.004*"forc" + 0.004*"peopl" + 0.004*"time" + 0.004*"like" + 0.004*"know" + 0.004*"militari" + 0.003*"citi" + 0.003*"state"
Topic: 4 
Words: 0.009*"peopl" + 0.007*"like" + 0.006*"world" + 0.005*"year" + 0.005*"time" + 0.004*"state" + 0.004*"go" + 0.004*"know" + 0.004*"american" + 0.004*"nation"
Topic: 5 
Words: 0.007*"peopl" + 0.006*"state" + 0.006*"govern" + 0.004*"year" + 0.004*"pipelin" + 0.004*"world" + 0.

In [29]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)

In [30]:
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))


Topic: 0 Word: 0.003*"clinton" + 0.002*"trump" + 0.002*"email" + 0.002*"hillari" + 0.001*"report" + 0.001*"state" + 0.001*"elect" + 0.001*"peopl" + 0.001*"year" + 0.001*"facebook"
Topic: 1 Word: 0.006*"clinton" + 0.006*"email" + 0.005*"investig" + 0.004*"hillari" + 0.004*"comey" + 0.003*"weiner" + 0.003*"trump" + 0.002*"abedin" + 0.002*"elect" + 0.002*"server"
Topic: 2 Word: 0.003*"dakota" + 0.003*"protest" + 0.003*"pipelin" + 0.002*"polic" + 0.002*"rock" + 0.002*"trump" + 0.002*"stand" + 0.002*"sioux" + 0.002*"sheriff" + 0.002*"arrest"
Topic: 3 Word: 0.003*"mosul" + 0.003*"syria" + 0.003*"aleppo" + 0.003*"syrian" + 0.003*"isi" + 0.002*"trump" + 0.002*"iraqi" + 0.002*"russian" + 0.002*"clinton" + 0.002*"terrorist"
Topic: 4 Word: 0.003*"trump" + 0.003*"vote" + 0.002*"clinton" + 0.002*"elect" + 0.002*"obama" + 0.002*"hillari" + 0.002*"state" + 0.001*"report" + 0.001*"presid" + 0.001*"voter"
Topic: 5 Word: 0.004*"trump" + 0.003*"clinton" + 0.003*"moreno" + 0.002*"hillari" + 0.002*"bundi" 

In [31]:

processed_docs[4310]

['common',
 'sens',
 'defin',
 'cambridg',
 'academ',
 'text',
 'dictionari',
 'abil',
 'good',
 'judgement',
 'make',
 'decis',
 'live',
 'reason',
 'safe',
 'sound',
 'reason',
 'safe',
 'sixti',
 'seventi',
 'grow',
 'risk',
 'daili',
 'base',
 'judgement',
 'decis',
 'last',
 'irrevers',
 'consequ',
 'start',
 'write',
 'sporad',
 'begin',
 'earli',
 'eighti',
 'small',
 'newspap',
 'like',
 'chicago',
 'daili',
 'news',
 'chicago',
 'defend',
 'enjoy',
 'opportun',
 'research',
 'expos',
 'resourc',
 'start',
 'canon',
 'career',
 'week',
 'contribut',
 'reader',
 'look',
 'career',
 'employ',
 'opportun',
 'financi',
 'case',
 'pay',
 'attend',
 'colleg',
 'start',
 'say',
 'guardian',
 'liberti',
 'voic',
 'motiv',
 'express',
 'person',
 'opinion',
 'world',
 'limit',
 'world',
 'view',
 'local',
 'landscap',
 'travail',
 'everyday',
 'world',
 'larg',
 'portray',
 'media',
 'mogul',
 'high',
 'pay',
 'infotain',
 'charlatan',
 'write',
 'vent',
 'frustrat',
 'fear',
 'world',


In [32]:
for index, score in sorted(lda_model[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.5360153317451477	 
Topic: 0.009*"peopl" + 0.007*"like" + 0.006*"world" + 0.005*"year" + 0.005*"time" + 0.004*"state" + 0.004*"go" + 0.004*"know" + 0.004*"american" + 0.004*"nation"

Score: 0.17714190483093262	 
Topic: 0.007*"peopl" + 0.006*"state" + 0.006*"govern" + 0.004*"year" + 0.004*"pipelin" + 0.004*"world" + 0.004*"time" + 0.004*"water" + 0.004*"know" + 0.004*"american"

Score: 0.11762698739767075	 
Topic: 0.006*"time" + 0.005*"like" + 0.005*"peopl" + 0.004*"year" + 0.004*"work" + 0.004*"know" + 0.003*"food" + 0.003*"bodi" + 0.003*"help" + 0.003*"video"

Score: 0.09282205253839493	 
Topic: 0.018*"trump" + 0.008*"elect" + 0.007*"american" + 0.007*"clinton" + 0.007*"presid" + 0.006*"peopl" + 0.006*"russia" + 0.006*"state" + 0.006*"govern" + 0.006*"syria"

Score: 0.04674949124455452	 
Topic: 0.011*"elect" + 0.011*"trump" + 0.011*"state" + 0.010*"vote" + 0.009*"clinton" + 0.006*"peopl" + 0.005*"presid" + 0.005*"voter" + 0.005*"obama" + 0.004*"time"

Score: 0.020118227228522

In [33]:
for index, score in sorted(lda_model_tfidf[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))


Score: 0.9714866280555725	 
Topic: 0.005*"trump" + 0.003*"clinton" + 0.003*"elect" + 0.002*"hillari" + 0.002*"vote" + 0.002*"russia" + 0.002*"american" + 0.002*"presid" + 0.002*"state" + 0.002*"peopl"

Score: 0.02568981982767582	 
Topic: 0.003*"clinton" + 0.002*"trump" + 0.002*"email" + 0.002*"hillari" + 0.001*"report" + 0.001*"state" + 0.001*"elect" + 0.001*"peopl" + 0.001*"year" + 0.001*"facebook"


In [34]:
unseen_document = 'How a Pentagon deal became an identity crisis for Google'
bow_vector = dictionary.doc2bow(preprocess(unseen_document))

for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.8498435616493225	 Topic: 0.018*"trump" + 0.008*"elect" + 0.007*"american" + 0.007*"clinton" + 0.007*"presid"
Score: 0.016692448407411575	 Topic: 0.011*"elect" + 0.011*"trump" + 0.011*"state" + 0.010*"vote" + 0.009*"clinton"
Score: 0.01668519340455532	 Topic: 0.009*"trump" + 0.007*"time" + 0.006*"like" + 0.006*"peopl" + 0.006*"think"
Score: 0.016684705391526222	 Topic: 0.006*"time" + 0.005*"like" + 0.005*"peopl" + 0.004*"year" + 0.004*"work"
Score: 0.01668364182114601	 Topic: 0.007*"peopl" + 0.006*"state" + 0.006*"govern" + 0.004*"year" + 0.004*"pipelin"
Score: 0.01668323017656803	 Topic: 0.024*"clinton" + 0.014*"hillari" + 0.011*"email" + 0.007*"investig" + 0.006*"campaign"
Score: 0.016682615503668785	 Topic: 0.007*"bank" + 0.006*"clinton" + 0.006*"world" + 0.006*"state" + 0.005*"gold"
Score: 0.016682488843798637	 Topic: 0.009*"peopl" + 0.007*"like" + 0.006*"world" + 0.005*"year" + 0.005*"time"
Score: 0.016681866720318794	 Topic: 0.009*"forc" + 0.009*"russian" + 0.008*"russia"